# 6. Группировка

### Библиотеки / данные

импортируем библиотеки numpy и pandas

In [1]:
import numpy as np
import pandas as pd

задаем некоторые настройки pandas, регулирующие формат вывода

In [2]:
pd.options.display.max_rows = 10

загружаем данные о счетах в ресторане (стоимость блюд, чаевые, курящий клиент, день недели, время дня, число блюд)

In [3]:
tips = pd.read_csv("data2/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


добавим столбец с относительным размером чаевых

In [4]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']

In [5]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


### Механизм GroupBy

<img src = 'images/split_apply_combine.png' style='width: 600px;'/>

In [6]:
df = pd.DataFrame({'x': ['a','a','b','b','c','c'],
                   'y': [2,4,0,5,5,10]})
df

,x,y
0,a,2
1,a,4
2,b,0
3,b,5
4,c,5
5,c,10


In [7]:
groups = df.groupby(['x'])
groups

получаем информацию о количестве групп, которые будут созданы

In [8]:
groups.ngroups

3

получаем информацию о количестве элементов в каждой группе

In [9]:
groups.size()

x
a    2
b    2
c    2
dtype: int64

что представляют из себя найденные группы?

In [10]:
groups.groups

{'a': [0, 1], 'b': [2, 3], 'c': [4, 5]}

получаем данные конкретной группы

In [11]:
groups.get_group('b')

,x,y
2,b,0
3,b,5


извлекаем первую (не нулевую) строку каждой группы

In [12]:
groups.nth([1])

,x,y
1,a,4
3,b,5
5,c,10


перебор всех групп:

In [13]:
for key, group in groups:
    print(key)
    print(group) 

('a',)
   x  y
0  a  2
1  a  4
('b',)
   x  y
2  b  0
3  b  5
('c',)
   x   y
4  c   5
5  c  10


вычисление среднего

In [17]:
groups.y.mean()

x
a    3.0
b    2.5
c    7.5
Name: y, dtype: float64

### Типы группировок

#### по столбцам: 

In [18]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


найдём средние чаевые на каждое время дня каждого дня в неделе 

In [19]:

tips.groupby(['day','time']).tip.mean()

day   time  
Fri   Dinner    2.940000
      Lunch     2.382857
Sat   Dinner    2.993103
Sun   Dinner    3.255132
Thur  Dinner    3.000000
      Lunch     2.767705
Name: tip, dtype: float64

или в компактной форме

In [20]:
tips.groupby(['day','time']).tip.mean().unstack()


time,Dinner,Lunch
day,,
Fri,2.940000,2.382857
Sat,2.993103,NaN
Sun,3.255132,NaN
Thur,3.000000,2.767705


#### по уровням индекса

создаем копию данных и заново индексируем ее

In [21]:
copy_tips = tips.copy()
copy_tips = copy_tips.set_index(['day', 'time'])
copy_tips

total_bill   tip smoker  size   tip_pct
day  time                                           
Sun  Dinner       16.99  1.01     No     2  0.059447
     Dinner       10.34  1.66     No     3  0.160542
     Dinner       21.01  3.50     No     3  0.166587
     Dinner       23.68  3.31     No     2  0.139780
     Dinner       24.59  3.61     No     4  0.146808
...                 ...   ...    ...   ...       ...
Sat  Dinner       29.03  5.92     No     3  0.203927
     Dinner       27.18  2.00    Yes     2  0.073584
     Dinner       22.67  2.00    Yes     2  0.088222
     Dinner       17.82  1.75     No     2  0.098204
Thur Dinner       18.78  3.00     No     2  0.159744

[244 rows x 5 columns]

группировать можем по одному или нескольким уровням индекса, передавая соответствующие значения столбцов аргументу level

In [22]:
copy_tips.groupby(level=['time']).sum()

,total_bill,tip,smoker,size,tip_pct
time,,,,,
Dinner,3660.30,546.07,NoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo...,463,28.075131
Lunch,1167.47,185.51,NoNoNoYesNoNoYesNoNoNoNoNoNoNoNoNoNoNoNoNoNoNo...,164,11.160699


In [23]:
copy_tips.groupby(level = ['day', 'time']).mean()

TypeError: agg function failed [how->mean,dtype->object]

#### с использованием функции 

In [24]:
copy_tips.head()

total_bill   tip smoker  size   tip_pct
day time                                           
Sun Dinner       16.99  1.01     No     2  0.059447
    Dinner       10.34  1.66     No     3  0.160542
    Dinner       21.01  3.50     No     3  0.166587
    Dinner       23.68  3.31     No     2  0.139780
    Dinner       24.59  3.61     No     4  0.146808

In [25]:
copy_tips.groupby(level='day').count()

,total_bill,tip,smoker,size,tip_pct
day,,,,,
Fri,19,19,19,19,19
Sat,87,87,87,87,87
Sun,76,76,76,76,76
Thur,62,62,62,62,62


In [26]:
tips.day.value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

#### с использованием массива 

создадим массив случайных меток групп той же длины, что и tips:

In [27]:
np.random.seed(123)
rnd_array = np.random.choice(['first_group', 'second_group'], 
                             size=tips.shape[0],
                             p = [0.4, 0.6])
rnd_array[0:5]


array(['second_group', 'first_group', 'first_group', 'second_group',
       'second_group'], dtype='<U12')

и передав его в метод groupby, определим сколько объектов соотнеслось с каждой меткой

In [28]:
tips.groupby(rnd_array).count()



,total_bill,tip,smoker,day,time,size,tip_pct
first_group,93,93,93,93,93,93,93
second_group,151,151,151,151,151,151,151


In [29]:
93/(93+151)

0.38114754098360654

с первой меткой почти 40% (как и просили)

## Дополнительные возможности манипуляций с группами

оценим частоту использования разных типов данных:

In [30]:
tips.dtypes

total_bill    float64
tip           float64
smoker         object
day            object
time           object
size            int64
tip_pct       float64
dtype: object

In [36]:
group_tips = tips.groupby(tips.dtypes)


In [34]:
group_tips.size()

Series([], dtype: int64)

### Агрегирование групп

создаем объект groupby:

In [37]:
group_tips = tips.groupby(['day', 'smoker'])['tip_pct']

используя собственную функцию оценки разброс относительной величины чаевых в каждой группе:

In [38]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

group_tips.agg([peak_to_peak])

peak_to_peak
day  smoker              
Fri  No          0.067349
     Yes         0.159925
Sat  No          0.235193
     Yes         0.290095
Sun  No          0.193226
     Yes         0.644685
Thur No          0.193350
     Yes         0.151240

комбинирование:

In [39]:
group_tips.agg([peak_to_peak, 'mean'])

peak_to_peak      mean
day  smoker                        
Fri  No          0.067349  0.151650
     Yes         0.159925  0.174783
Sat  No          0.235193  0.158048
     Yes         0.290095  0.147906
Sun  No          0.193226  0.160113
     Yes         0.644685  0.187250
Thur No          0.193350  0.160298
     Yes         0.151240  0.163863

задаем имена столбцов:

In [40]:
group_tips.agg([('delta_max_min', peak_to_peak), ('mean_value','mean')])

delta_max_min  mean_value
day  smoker                           
Fri  No           0.067349    0.151650
     Yes          0.159925    0.174783
Sat  No           0.235193    0.158048
     Yes          0.290095    0.147906
Sun  No           0.193226    0.160113
     Yes          0.644685    0.187250
Thur No           0.193350    0.160298
     Yes          0.151240    0.163863

отдельные наборы функций для каждого столбца:

In [41]:
group_tips = tips.groupby(['day', 'smoker'])['tip_pct', 'total_bill']

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [42]:
#group_tips.agg({'tip_pct':[('max_value', np.max),
group_tips.agg({'tip_pct':[('max_value','max'),
                           ('min_value',np.min)],
                'total_bill': 'sum'})

SpecificationError: nested renamer is not supported

### Преобразование групп

#### метод transform

<img src = 'images/transform.png' style='width: 900px;'/>

In [47]:
df = pd.DataFrame({'Col1': ['A', 'B', 'C', 'C', 'B', 'B', 'A'],
                   'Col2': [1, 2, 3, 4, 2, 5, 3]})
df

,Col1,Col2
0,A,1
1,B,2
2,C,3
3,C,4
4,B,2
5,B,5
6,A,3


In [48]:
df['Col3'] = df.groupby('Col1').transform(sum)
df.sort_values('Col1')

C:\Users\vik\AppData\Local\Temp\ipykernel_9072\3950718050.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df['Col3'] = df.groupby('Col1').transform(sum)


,Col1,Col2,Col3
0,A,1,4
6,A,3,4
1,B,2,9
4,B,2,9
5,B,5,9
2,C,3,7
3,C,4,7


объект groupby:

In [49]:
group_tips = tips.groupby(['smoker'])['total_bill']
group_tips


нормирование:

In [50]:
norm = lambda x: (x - x.mean())/x.std()
group_tips.transform(norm)

0     -0.266278
1     -1.071793
2      0.220665
3      0.544083
4      0.654311
         ...   
239    1.192129
240    0.653331
241    0.194632
242   -0.165740
243   -0.049455
Name: total_bill, Length: 244, dtype: float64

In [51]:
tips_copy = tips.copy()
tips_copy['total_bill_norm'] = group_tips.transform(norm)
tips_copy.head()

,total_bill,tip,smoker,day,time,size,tip_pct,total_bill_norm
0,16.99,1.01,No,Sun,Dinner,2,0.059447,-0.266278
1,10.34,1.66,No,Sun,Dinner,3,0.160542,-1.071793
2,21.01,3.50,No,Sun,Dinner,3,0.166587,0.220665
3,23.68,3.31,No,Sun,Dinner,2,0.139780,0.544083
4,24.59,3.61,No,Sun,Dinner,4,0.146808,0.654311


#### метод apply

функция:

In [52]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


применяем метод apply

In [53]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

### Исключение групп

создаем данные для наших примеров

In [54]:
df = pd.DataFrame({'Label': list('AABCCC'),
                   'Values': [1, 2, 3, 4, np.nan, 8]})
df

,Label,Values
0,A,1.0
1,A,2.0
2,B,3.0
3,C,4.0
4,C,NaN
5,C,8.0


удаляем группы с менее чем двумя возможными значениеми

In [55]:
f = lambda x: x.Values.count() > 1
df.groupby('Label').filter(f)

,Label,Values
0,A,1.0
1,A,2.0
3,C,4.0
4,C,NaN
5,C,8.0


удаляем группы, в которых есть пропуски

In [56]:
f = lambda x: x.Values.isnull().sum() == 0
df.groupby('Label').filter(f)

,Label,Values
0,A,1.0
1,A,2.0
2,B,3.0


### Сводная таблица

средние

In [57]:
tips.pivot_table(index=['day', 'smoker'])

TypeError: agg function failed [how->mean,dtype->object]

вывод по строкам и столбцам

In [58]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

включение частичных итогов:

In [59]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

задаем функцию:

In [60]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=max, margins=True)

C:\Users\vik\AppData\Local\Temp\ipykernel_9072\4192889745.py:1: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
C:\Users\vik\AppData\Local\Temp\ipykernel_9072\4192889745.py:1: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
C:\Users\vik\AppData\Local\Temp\ipykernel_9072\4192889745.py:1: FutureWarning: The provided callable <built-in function max> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current beha

day                 Fri       Sat       Sun      Thur       All
time   smoker                                                  
Dinner No      0.155625  0.291990  0.252672  0.159744  0.291990
       Yes     0.263480  0.325733  0.710345       NaN  0.710345
Lunch  No      0.187735       NaN       NaN  0.266312  0.266312
       Yes     0.259314       NaN       NaN  0.241255  0.259314
All            0.263480  0.325733  0.710345  0.266312  0.710345